In [1002]:
import pandas as pd
import numpy as np
import os
from calendar import monthrange
import time

# 1. Preparing Data <a id="introduction"></a>

## 1.1 Loading Data

In [1003]:
forecast_output_file_path = r'C:\Users\Anamind\Downloads\NDC\Client5_ForecastOutput.csv'
df_forecast_output = pd.read_csv(forecast_output_file_path, usecols=['Item Code',	'Location',	'Channel',	'Month', 'Best Fit Forecast',	'First Forecast Month',	'Override'])

item_info_file_path = r'C:\Users\Anamind\Downloads\NDC\Client5_Iteminfo.csv'
df_item_info = pd.read_csv(item_info_file_path, encoding='ISO-8859-1', usecols=['Item Code', 'Location', 'Last on hand', 'Lead time', 'Safety Stock Cover', 'Units per pallet', 'Shelf life, days', 'Min Shelf Life', 'DC Name'])


batch_expiry_file_path = r'C:\Users\Anamind\Downloads\NDC\Client5_Batchexpiry.csv'
df_batch_expiry = pd.read_csv(batch_expiry_file_path, usecols=['Item Code', 'Location', 'Expiry Date', 'Expiry Quantity'])

intransit_file_path = r'C:\Users\Anamind\Downloads\NDC\Client5_Intransit.csv'
df_intransit = pd.read_csv(intransit_file_path, usecols=['Item Code', 'Location','Delivery date', 'Qty to receive'])



In [1004]:
df_batch_expiry

,Item Code,Location,Expiry Date,Expiry Quantity
0,21150,Amazon FBA,31-07-2024,44


In [1005]:
df_item_info

,Item Code,Location,Last on hand,Lead time,Safety Stock Cover,Units per pallet,"Shelf life, days",Min Shelf Life,DC Name
0,21150,Bhiwandi,381,2,6,40,120,NaN,NaN
1,21351,Bhiwandi,0,2,6,40,120,NaN,NaN
2,21560,Bhiwandi,80,2,4,40,120,NaN,NaN
3,21900,Bhiwandi,148,2,4,40,120,NaN,NaN
4,21907,Bhiwandi,248,2,4,40,120,NaN,NaN
...,...,...,...,...,...,...,...,...,...
330,7.76E+11,Aramax Bangalore,0,1,2,20,120,NaN,NaN
331,PCSLBG11023,Bhiwandi,0,2,4,20,120,NaN,NaN
332,7.76E+11,Bhiwandi,0,2,4,20,120,NaN,NaN
333,PCSLBG11023,Aramax Bangalore,0,1,2,20,120,NaN,NaN


In [1006]:
df_item_info.fillna({
    'Last on Hand': 0,
    'Lead time': 1,
    'Safety Stock Cover': 1,
    'Units per pallet': 1,
    'Min Shelf Life':0,
    'Shelf Life':720
}, inplace=True)

Let us look at the sample data in each file.

In [1007]:
df_batch_expiry['Item Code'] = df_batch_expiry['Item Code'].astype(str)

In [1008]:
df_batch_expiry['Item Code'] = df_batch_expiry['Item Code'].astype(str)
df_item_info['Item Code'] = df_item_info['Item Code'].astype(str)

In [1009]:
df_batch_expiry['Expiry Date'] = pd.to_datetime(df_batch_expiry['Expiry Date'], format='%d-%m-%Y')

# Merge the DataFrames on 'Item Code'
df_batch_expiry = pd.merge(df_batch_expiry, df_item_info, on=['Item Code','Location'])

# Subtract 'Min shelf' days from 'Expiry Date'
df_batch_expiry['Expiry Date'] = df_batch_expiry['Expiry Date'] - pd.to_timedelta(df_batch_expiry['Min Shelf Life'], unit='days')

# Convert 'New Date' to the desired format
#df_batch_expiry['Expiry Date'] = df_batch_expiry['Expiry Date'].dt.strftime('%d-%m-%Y')

df_batch_expiry = df_batch_expiry[['Item Code', 'Location',  'Expiry Date', 'Expiry Quantity']]

# Rename the column Location_x to Location
#df_batch_expiry.rename(columns={'Location': 'Location'}, inplace=True)

In [1010]:
df_item_info['Shelf life, days'] = df_item_info['Shelf life, days'] - df_item_info['Min Shelf Life']
df_item_info['Shelf life, days'] = df_item_info['Shelf life, days'].apply(lambda x: max(x, 0))


In [1011]:
df_batch_expiry

,Item Code,Location,Expiry Date,Expiry Quantity
0,21150,Amazon FBA,2024-07-31,44


In [1012]:
df_item_info.head(5)

,Item Code,Location,Last on hand,Lead time,Safety Stock Cover,Units per pallet,"Shelf life, days",Min Shelf Life,DC Name
0,21150,Bhiwandi,381,2,6,40,120.0,0.0,NaN
1,21351,Bhiwandi,0,2,6,40,120.0,0.0,NaN
2,21560,Bhiwandi,80,2,4,40,120.0,0.0,NaN
3,21900,Bhiwandi,148,2,4,40,120.0,0.0,NaN
4,21907,Bhiwandi,248,2,4,40,120.0,0.0,NaN


In [1013]:
df_batch_expiry.head(5)

,Item Code,Location,Expiry Date,Expiry Quantity
0,21150,Amazon FBA,2024-07-31,44


In [1014]:
df_intransit.head(5)

,Item Code,Location,Delivery date,Qty to receive
0,23807,Bhiwandi,31-05-2024,96
1,25700,Bhiwandi,31-05-2024,240
2,25707,Bhiwandi,31-05-2024,240
3,26130,Bhiwandi,31-05-2024,48
4,27600,Bhiwandi,31-05-2024,48


Let us convert all the dates in one common format to maintain consitency and easy of computation. The format we will follow is YYYY-MM-01. All the calculations will be based on 1st of every month. 

In [1015]:
# Convert 'Expiry Date' column to datetime without specifying format (assuming no warnings here)
df_batch_expiry['Expiry Date'] = pd.to_datetime(df_batch_expiry['Expiry Date'])

# Convert 'Month' column with explicit format
df_forecast_output['Month'] = pd.to_datetime(df_forecast_output['Month'], format='%d-%m-%Y').dt.strftime('%Y-%m')

# Convert 'First Forecast Month' column with explicit format
df_forecast_output['First Forecast Month'] = pd.to_datetime(df_forecast_output['First Forecast Month'], format='%d-%m-%Y').dt.strftime('%Y-%m')

# Convert 'Delivery date' column with explicit format
df_intransit['Delivery date'] = pd.to_datetime(df_intransit['Delivery date'], format='%d-%m-%Y').dt.strftime('%Y-%m')


In [1016]:
df_intransit

,Item Code,Location,Delivery date,Qty to receive
0,23807,Bhiwandi,2024-05,96
1,25700,Bhiwandi,2024-05,240
2,25707,Bhiwandi,2024-05,240
3,26130,Bhiwandi,2024-05,48
4,27600,Bhiwandi,2024-05,48
5,27691,Bhiwandi,2024-05,120
6,27750,Bhiwandi,2024-05,520
7,27800,Bhiwandi,2024-05,336
8,27807,Bhiwandi,2024-05,260
9,27830,Bhiwandi,2024-05,192


In [1017]:
df_batch_expiry['Item Code'] = df_batch_expiry['Item Code'].astype(str)
df_batch_expiry['Location'] = df_batch_expiry['Location'].astype(str)

df_forecast_output['Item Code'] = df_forecast_output['Item Code'].astype(str)
df_forecast_output['Location'] = df_forecast_output['Location'].astype(str)

df_intransit['Item Code'] = df_intransit['Item Code'].astype(str)
df_intransit['Location'] = df_intransit['Location'].astype(str)

df_item_info['Item Code'] = df_item_info['Item Code'].astype(str)
df_item_info['Location'] = df_item_info['Location'].astype(str)

# 2. Computations

## 2.1- Creating Final Forecast

We need to replace the 'Best Fit Forecast' with 'Override' data when 'Override' data is available. Let us look at sample data where 'Override' column is not NA.

In [1018]:
np_forecast_output = df_forecast_output.to_numpy()
def combine_columns_numpy(row):
    return row[6] if pd.notna(row[6]) else row[4]

# Create a boolean mask to identify rows where column 6 is not NaN
not_na_mask = pd.notna(np_forecast_output[:, 6])

# Initialize a new array with values from column 6 where not NaN, otherwise from column 4
combined_column = np.where(not_na_mask, np_forecast_output[:, 6], np_forecast_output[:, 4])

# Replace 'Best Fit Forecast' column in original DataFrame with the combined values
df_forecast_output['Best Fit Forecast'] = combined_column
df_forecast_output.head(10)

,Item Code,Location,Channel,Month,Best Fit Forecast,First Forecast Month,Override
0,21150,Amazon FBA,Amazon,2024-05,0.73,2024-05,NaN
1,21150,Amazon FBA,Amazon,2024-06,0.97,2024-05,NaN
2,21150,Amazon FBA,Amazon,2024-07,0.97,2024-05,NaN
3,21150,Amazon FBA,Amazon,2024-08,1.06,2024-05,NaN
4,21150,Amazon FBA,Amazon,2024-09,1.06,2024-05,NaN
5,21150,Amazon FBA,Amazon,2024-10,1.06,2024-05,NaN
6,21150,Amazon FBA,Amazon,2024-11,1.09,2024-05,NaN
7,21150,Amazon FBA,Amazon,2024-12,1.12,2024-05,NaN
8,21150,Amazon FBA,Amazon,2025-01,1.09,2024-05,NaN
9,21150,Amazon FBA,Amazon,2025-02,1.15,2024-05,NaN


In [1019]:
df_forecast_output = df_forecast_output.drop(columns=['Override'])
df_forecast_output

,Item Code,Location,Channel,Month,Best Fit Forecast,First Forecast Month
0,21150,Amazon FBA,Amazon,2024-05,0.73,2024-05
1,21150,Amazon FBA,Amazon,2024-06,0.97,2024-05
2,21150,Amazon FBA,Amazon,2024-07,0.97,2024-05
3,21150,Amazon FBA,Amazon,2024-08,1.06,2024-05
4,21150,Amazon FBA,Amazon,2024-09,1.06,2024-05
...,...,...,...,...,...,...
9895,PCSLBG11023,Bhiwandi,Brandstore - D2C,2025-06,0.0,2024-05
9896,PCSLBG11023,Bhiwandi,Brandstore - D2C,2025-07,0.0,2024-05
9897,PCSLBG11023,Bhiwandi,Brandstore - D2C,2025-08,0.0,2024-05
9898,PCSLBG11023,Bhiwandi,Brandstore - D2C,2025-09,0.0,2024-05


In [1020]:
df_forecast_output= df_forecast_output.groupby(['Month', 'Item Code', 'Location','First Forecast Month'])['Best Fit Forecast'].sum().reset_index()

In [1021]:
df_first_forecast = df_forecast_output.drop(columns=[ 'Month', 'Best Fit Forecast']).drop_duplicates()
df_first_forecast

,Item Code,Location,First Forecast Month
0,19076008,Aramax Bangalore,2024-05
1,19076008,Bhiwandi,2024-05
2,2.01E+11,Aramax Bangalore,2024-05
3,2.01E+11,Bhiwandi,2024-05
4,2.01E+12,Bhiwandi,2024-05
...,...,...,...
223,93030,Bhiwandi,2024-05
224,93030,Myntra SJIT - Bangalore,2024-05
225,93030,Myntra SJIT - Mumbai,2024-05
226,PCSLBG11023,Aramax Bangalore,2024-05


In [1022]:
df_forecast_output = df_forecast_output.drop(columns=['First Forecast Month'])
df_forecast_output

,Month,Item Code,Location,Best Fit Forecast
0,2024-05,19076008,Aramax Bangalore,0.0
1,2024-05,19076008,Bhiwandi,0.0
2,2024-05,2.01E+11,Aramax Bangalore,0.5
3,2024-05,2.01E+11,Bhiwandi,0.5
4,2024-05,2.01E+12,Bhiwandi,0.57
...,...,...,...,...
4099,2025-10,93030,Bhiwandi,4.48
4100,2025-10,93030,Myntra SJIT - Bangalore,0.43
4101,2025-10,93030,Myntra SJIT - Mumbai,0.0
4102,2025-10,PCSLBG11023,Aramax Bangalore,0.0


## 2.2 Process Item Info

In [1023]:
df_item_info

,Item Code,Location,Last on hand,Lead time,Safety Stock Cover,Units per pallet,"Shelf life, days",Min Shelf Life,DC Name
0,21150,Bhiwandi,381,2,6,40,120.0,0.0,NaN
1,21351,Bhiwandi,0,2,6,40,120.0,0.0,NaN
2,21560,Bhiwandi,80,2,4,40,120.0,0.0,NaN
3,21900,Bhiwandi,148,2,4,40,120.0,0.0,NaN
4,21907,Bhiwandi,248,2,4,40,120.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...
330,7.76E+11,Aramax Bangalore,0,1,2,20,120.0,0.0,NaN
331,PCSLBG11023,Bhiwandi,0,2,4,20,120.0,0.0,NaN
332,7.76E+11,Bhiwandi,0,2,4,20,120.0,0.0,NaN
333,PCSLBG11023,Aramax Bangalore,0,1,2,20,120.0,0.0,NaN


In [1024]:
df_intransit

,Item Code,Location,Delivery date,Qty to receive
0,23807,Bhiwandi,2024-05,96
1,25700,Bhiwandi,2024-05,240
2,25707,Bhiwandi,2024-05,240
3,26130,Bhiwandi,2024-05,48
4,27600,Bhiwandi,2024-05,48
5,27691,Bhiwandi,2024-05,120
6,27750,Bhiwandi,2024-05,520
7,27800,Bhiwandi,2024-05,336
8,27807,Bhiwandi,2024-05,260
9,27830,Bhiwandi,2024-05,192


In [1025]:
# Select all necessary columns in one go
selected_columns = ['Item Code', 'Location', 'Last on hand', 'Shelf life, days', 'Min Shelf Life']

df_selected = df_item_info[selected_columns]


# Create individual DataFrames
#df_groups = df_selected[['Item Code', 'Location']]
df_shelf_life = df_selected[['Item Code', 'Location', 'Shelf life, days', 'Last on hand']]
df_loh = df_selected[['Item Code', 'Location', 'Last on hand']]


#df_min_shelf_life = df_selected[['Item Code', 'Location', 'Min Shelf Life']]

#df_min_shelf_life.fillna(365*200)

#print(df_groups)
#print(df_shelf_life)
#print(df_loh)
#print(df_min_shelf_life)

In [1026]:
df_shelf_life

,Item Code,Location,"Shelf life, days",Last on hand
0,21150,Bhiwandi,120.0,381
1,21351,Bhiwandi,120.0,0
2,21560,Bhiwandi,120.0,80
3,21900,Bhiwandi,120.0,148
4,21907,Bhiwandi,120.0,248
...,...,...,...,...
330,7.76E+11,Aramax Bangalore,120.0,0
331,PCSLBG11023,Bhiwandi,120.0,0
332,7.76E+11,Bhiwandi,120.0,0
333,PCSLBG11023,Aramax Bangalore,120.0,0


In [1027]:
df_intransit

,Item Code,Location,Delivery date,Qty to receive
0,23807,Bhiwandi,2024-05,96
1,25700,Bhiwandi,2024-05,240
2,25707,Bhiwandi,2024-05,240
3,26130,Bhiwandi,2024-05,48
4,27600,Bhiwandi,2024-05,48
5,27691,Bhiwandi,2024-05,120
6,27750,Bhiwandi,2024-05,520
7,27800,Bhiwandi,2024-05,336
8,27807,Bhiwandi,2024-05,260
9,27830,Bhiwandi,2024-05,192


In [1028]:
df_item_info

,Item Code,Location,Last on hand,Lead time,Safety Stock Cover,Units per pallet,"Shelf life, days",Min Shelf Life,DC Name
0,21150,Bhiwandi,381,2,6,40,120.0,0.0,NaN
1,21351,Bhiwandi,0,2,6,40,120.0,0.0,NaN
2,21560,Bhiwandi,80,2,4,40,120.0,0.0,NaN
3,21900,Bhiwandi,148,2,4,40,120.0,0.0,NaN
4,21907,Bhiwandi,248,2,4,40,120.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...
330,7.76E+11,Aramax Bangalore,0,1,2,20,120.0,0.0,NaN
331,PCSLBG11023,Bhiwandi,0,2,4,20,120.0,0.0,NaN
332,7.76E+11,Bhiwandi,0,2,4,20,120.0,0.0,NaN
333,PCSLBG11023,Aramax Bangalore,0,1,2,20,120.0,0.0,NaN


In [1029]:
df_item_info = df_item_info.drop(columns=['Last on hand', 'Min Shelf Life'])
df_item_info

,Item Code,Location,Lead time,Safety Stock Cover,Units per pallet,"Shelf life, days",DC Name
0,21150,Bhiwandi,2,6,40,120.0,NaN
1,21351,Bhiwandi,2,6,40,120.0,NaN
2,21560,Bhiwandi,2,4,40,120.0,NaN
3,21900,Bhiwandi,2,4,40,120.0,NaN
4,21907,Bhiwandi,2,4,40,120.0,NaN
...,...,...,...,...,...,...,...
330,7.76E+11,Aramax Bangalore,1,2,20,120.0,NaN
331,PCSLBG11023,Bhiwandi,2,4,20,120.0,NaN
332,7.76E+11,Bhiwandi,2,4,20,120.0,NaN
333,PCSLBG11023,Aramax Bangalore,1,2,20,120.0,NaN


## 2.3 Batch Expiry

In [1030]:
df_batch_expiry

,Item Code,Location,Expiry Date,Expiry Quantity
0,21150,Amazon FBA,2024-07-31,44


In [1031]:

#df_batch_expiry['Expiry Date'] = df_batch_expiry['Expiry Date'] - pd.DateOffset(months=3)
df_batch_expiry['Expiry Date'] = df_batch_expiry['Expiry Date'].dt.strftime('%Y-%m')
# Rename 'Expiry Date' column to 'Month'
df_batch_expiry.rename(columns={'Expiry Date': 'Month'}, inplace=True)
df_batch_expiry

,Item Code,Location,Month,Expiry Quantity
0,21150,Amazon FBA,2024-07,44


In [1032]:
df_batch_expiry = df_batch_expiry.groupby(['Item Code', 'Location','Month'], as_index=False).agg({
    'Expiry Quantity': 'sum'
})

In [1033]:
print(df_batch_expiry['Month'].head())
print(df_batch_expiry['Month'].dtype)

0    2024-07
Name: Month, dtype: object
object


In [1034]:
df_unique_groups = df_batch_expiry [['Item Code', 'Location']].drop_duplicates()
df_batch_expiry['Month'] = pd.to_datetime(df_batch_expiry['Month'], errors='coerce')

# Now you can use .dt.strftime('%Y-%m')
df_batch_expiry['Month'] = df_batch_expiry['Month'].astype(str)

# Now proceed with the rest of your operations
numeric_cols = ['Expiry Quantity']
df_batch_expiry[numeric_cols] = df_batch_expiry[numeric_cols].round(2).fillna(0).astype(int)

# Check the final result
#alll columns in numpy
item_codes = df_batch_expiry['Item Code'].to_numpy()
locations = df_batch_expiry ['Location'].to_numpy()
expiry_dates =df_batch_expiry ['Month'].to_numpy()
expired_quantities = df_batch_expiry['Expiry Quantity'].to_numpy()

# Initialize nested array to hold grouped data
dict_month_level_expiry = {}

# Iterate through unique combinations and group corresponding rows
for i, (item_code, location) in enumerate(df_unique_groups.itertuples(index=False)):
    mask = (item_codes == item_code) & (locations == location)
    group_data = np.column_stack((
        expiry_dates[mask],
        expired_quantities[mask]
    ))
    dict_month_level_expiry[(item_code, location)] = group_data

#print dictionary
for key, value in dict_month_level_expiry.items():
    print(f"Key: {key}")
    print(f"Value:\n{value}\n")

Key: ('21150', 'Amazon FBA')
Value:
[['2024-07-01' 44]]



In [1035]:
df_batch_expiry

,Item Code,Location,Month,Expiry Quantity
0,21150,Amazon FBA,2024-07-01,44


In [1036]:
df_batch_expiry = pd.merge(df_first_forecast, df_batch_expiry, on=['Item Code', 'Location'], how='left')


In [1037]:
df_batch_expiry

,Item Code,Location,First Forecast Month,Month,Expiry Quantity
0,19076008,Aramax Bangalore,2024-05,NaN,NaN
1,19076008,Bhiwandi,2024-05,NaN,NaN
2,2.01E+11,Aramax Bangalore,2024-05,NaN,NaN
3,2.01E+11,Bhiwandi,2024-05,NaN,NaN
4,2.01E+12,Bhiwandi,2024-05,NaN,NaN
...,...,...,...,...,...
223,93030,Bhiwandi,2024-05,NaN,NaN
224,93030,Myntra SJIT - Bangalore,2024-05,NaN,NaN
225,93030,Myntra SJIT - Mumbai,2024-05,NaN,NaN
226,PCSLBG11023,Aramax Bangalore,2024-05,NaN,NaN


In [1038]:
# Merge the DataFrames
df_batch_expiry = pd.merge(df_batch_expiry, df_shelf_life, on=['Item Code', 'Location'], how='left')


df_batch_expiry['Month'] = pd.to_datetime(df_batch_expiry['Month'], errors='coerce')

# Convert columns to NumPy arrays
first_forecast_month = df_batch_expiry['First Forecast Month'].to_numpy(dtype='datetime64[ns]')
month = df_batch_expiry['Month'].to_numpy()
shelf_life_days = df_batch_expiry['Shelf life, days'].to_numpy()
exp_qty = df_batch_expiry['Expiry Quantity']
loh = df_batch_expiry['Last on hand']

# Identify NaN values in 'Month'
nan_mask = pd.isnull(month)

# Calculate new 'Month' values for NaNs
new_month_values = first_forecast_month[nan_mask] + pd.to_timedelta(shelf_life_days[nan_mask], unit='d')

# Update 'Month' column with the new values
month[nan_mask] = new_month_values

# Assign the updated 'Month' values back to the DataFrame
df_batch_expiry['Month'] = month

df_batch_expiry.loc[nan_mask, 'Expiry Quantity'] = loh[nan_mask]

# Drop the 'Shelf life, days' column if not needed anymore
df_batch_expiry.drop(columns=['Shelf life, days', 'Last on hand'], inplace=True)

# Display the result
print(df_batch_expiry)

       Item Code                 Location First Forecast Month      Month  \
0       19076008         Aramax Bangalore              2024-05 2024-08-29   
1       19076008                 Bhiwandi              2024-05 2024-08-29   
2       2.01E+11         Aramax Bangalore              2024-05 2024-08-29   
3       2.01E+11         Aramax Bangalore              2024-05 2024-08-29   
4       2.01E+11                 Bhiwandi              2024-05 2024-08-29   
..           ...                      ...                  ...        ...   
227        93030                 Bhiwandi              2024-05 2024-08-29   
228        93030  Myntra SJIT - Bangalore              2024-05 2024-08-29   
229        93030     Myntra SJIT - Mumbai              2024-05 2024-08-29   
230  PCSLBG11023         Aramax Bangalore              2024-05 2024-08-29   
231  PCSLBG11023                 Bhiwandi              2024-05 2024-08-29   

     Expiry Quantity  
0                0.0  
1                0.0  
2     

In [1039]:
# Convert 'First Forecast Month' and 'Month' to datetime
df_batch_expiry['First Forecast Month'] = pd.to_datetime(df_batch_expiry['First Forecast Month']).dt.strftime('%Y-%m')
df_batch_expiry['Month'] = pd.to_datetime(df_batch_expiry['Month']).dt.strftime('%Y-%m')
df_batch_expiry['Month'] = df_batch_expiry[['Month', 'First Forecast Month']].max(axis=1)
df_batch_expiry

,Item Code,Location,First Forecast Month,Month,Expiry Quantity
0,19076008,Aramax Bangalore,2024-05,2024-08,0.0
1,19076008,Bhiwandi,2024-05,2024-08,0.0
2,2.01E+11,Aramax Bangalore,2024-05,2024-08,0.0
3,2.01E+11,Aramax Bangalore,2024-05,2024-08,0.0
4,2.01E+11,Bhiwandi,2024-05,2024-08,0.0
...,...,...,...,...,...
227,93030,Bhiwandi,2024-05,2024-08,83.0
228,93030,Myntra SJIT - Bangalore,2024-05,2024-08,0.0
229,93030,Myntra SJIT - Mumbai,2024-05,2024-08,0.0
230,PCSLBG11023,Aramax Bangalore,2024-05,2024-08,0.0


In [1040]:
df_loh

,Item Code,Location,Last on hand
0,21150,Bhiwandi,381
1,21351,Bhiwandi,0
2,21560,Bhiwandi,80
3,21900,Bhiwandi,148
4,21907,Bhiwandi,248
...,...,...,...
330,7.76E+11,Aramax Bangalore,0
331,PCSLBG11023,Bhiwandi,0
332,7.76E+11,Bhiwandi,0
333,PCSLBG11023,Aramax Bangalore,0


In [1041]:
item_codes = df_loh['Item Code'].values
locations = df_loh['Location'].values
last_on_hand_values = df_loh['Last on hand'].values.astype(int)

# Create dictionary using NumPy arrays
dict_loh = {(item_code, location): last_on_hand_value for item_code, location, last_on_hand_value in zip(item_codes, locations, last_on_hand_values)}


In [1042]:
df_batch_expiry = df_batch_expiry.drop(columns=['First Forecast Month'])
df_exp_qty = df_batch_expiry.groupby(['Item Code', 'Location'], as_index=False).agg({
    'Expiry Quantity': 'sum'
})
df_exp_qty_with_date = df_batch_expiry.groupby(['Item Code', 'Location', 'Month'], as_index=False).agg({
    'Expiry Quantity': 'sum'
})
df_exp_qty_with_date

,Item Code,Location,Month,Expiry Quantity
0,19076008,Aramax Bangalore,2024-08,0.0
1,19076008,Bhiwandi,2024-08,0.0
2,2.01E+11,Aramax Bangalore,2024-08,0.0
3,2.01E+11,Bhiwandi,2024-08,0.0
4,2.01E+12,Bhiwandi,2024-08,0.0
...,...,...,...,...
223,93030,Bhiwandi,2024-08,83.0
224,93030,Myntra SJIT - Bangalore,2024-08,0.0
225,93030,Myntra SJIT - Mumbai,2024-08,0.0
226,PCSLBG11023,Aramax Bangalore,2024-08,0.0


# 2.4 Shipment

In [1043]:
df_month_level_receive = pd.merge(df_intransit, df_item_info, on=['Item Code', 'Location'], how='left')

In [1044]:
# Rename 'Expiry Date' column to 'Month'
df_intransit.rename(columns={'Delivery date': 'Month'}, inplace=True)
df_intransit

,Item Code,Location,Month,Qty to receive
0,23807,Bhiwandi,2024-05,96
1,25700,Bhiwandi,2024-05,240
2,25707,Bhiwandi,2024-05,240
3,26130,Bhiwandi,2024-05,48
4,27600,Bhiwandi,2024-05,48
5,27691,Bhiwandi,2024-05,120
6,27750,Bhiwandi,2024-05,520
7,27800,Bhiwandi,2024-05,336
8,27807,Bhiwandi,2024-05,260
9,27830,Bhiwandi,2024-05,192


In [1045]:
df_intransit = pd.merge(df_first_forecast, df_intransit, on=['Item Code', 'Location'], how='right')
df_intransit

,Item Code,Location,First Forecast Month,Month,Qty to receive
0,23807,Bhiwandi,2024-05,2024-05,96
1,25700,Bhiwandi,NaN,2024-05,240
2,25707,Bhiwandi,NaN,2024-05,240
3,26130,Bhiwandi,NaN,2024-05,48
4,27600,Bhiwandi,2024-05,2024-05,48
5,27691,Bhiwandi,2024-05,2024-05,120
6,27750,Bhiwandi,2024-05,2024-05,520
7,27800,Bhiwandi,NaN,2024-05,336
8,27807,Bhiwandi,NaN,2024-05,260
9,27830,Bhiwandi,2024-05,2024-05,192


In [1046]:
df_intransit['Month'] = pd.to_datetime(df_intransit['Month'], errors='coerce')
df_intransit['First Forecast Month'] = pd.to_datetime(df_intransit['First Forecast Month'], errors='coerce')

# Now apply the max function
df_intransit['Month'] = df_intransit[['Month', 'First Forecast Month']].max(axis=1)

In [1047]:
df_intransit = df_intransit.drop(columns=['First Forecast Month'])
df_intransit = df_intransit.groupby(['Item Code', 'Location', 'Month'], as_index=False).agg({
    'Qty to receive': 'sum'
})
df_intransit

,Item Code,Location,Month,Qty to receive
0,23807,Bhiwandi,2024-05-01,96
1,25700,Bhiwandi,2024-05-01,240
2,25707,Bhiwandi,2024-05-01,240
3,26130,Bhiwandi,2024-05-01,48
4,27600,Bhiwandi,2024-05-01,48
5,27691,Bhiwandi,2024-05-01,120
6,27710,Bhiwandi,2024-05-01,144
7,27750,Bhiwandi,2024-05-01,520
8,27800,Bhiwandi,2024-05-01,336
9,27807,Bhiwandi,2024-05-01,260


In [1048]:
df_month_level_receive = pd.merge(df_intransit, df_item_info, on=['Item Code', 'Location'], how='left')

In [1049]:
df_month_level_receive = df_month_level_receive.rename(columns={'Month': 'Delivery date'})

In [1050]:
df_month_level_receive['Delivery date'] = pd.to_datetime(df_month_level_receive['Delivery date'])
df_month_level_receive['Shelf life, days'].fillna(720, inplace=True)

# Calculate shelf life in months and round up
shelf_life_months_np = np.ceil(df_month_level_receive['Shelf life, days'] / 30).astype(int)

# Convert months to days and add as timedelta
expiry_date_np = df_month_level_receive['Delivery date'] + pd.to_timedelta(shelf_life_months_np * 30 + 30, unit='D')

# Ensure expiry_date_np is a datetime object
#print(expiry_date_np.dtype)  # Should be datetime64[ns]

# Format expiry date to YYYY/MM format
df_month_level_receive['Expiry Date'] = expiry_date_np.dt.to_period('M').astype(str)

C:\Users\Anamind\AppData\Local\Temp\ipykernel_21924\1226410733.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_month_level_receive['Shelf life, days'].fillna(720, inplace=True)


In [1051]:
df_month_level_receive['Expiry Date'] = pd.to_datetime(df_month_level_receive['Expiry Date'], format='%Y-%m')
df_month_level_receive['Expiry Date'] = df_month_level_receive['Expiry Date'].dt.strftime('%Y-%m')

#time format for numpy
df_month_level_receive['Delivery date'] = pd.to_datetime(df_month_level_receive['Delivery date'], format='%Y-%m-%d')
df_month_level_receive['Delivery date'] = df_month_level_receive['Delivery date'].dt.strftime('%Y-%m')


#get unique item groups
df_unique_groups_expiry = df_month_level_receive[['Item Code', 'Location']].drop_duplicates()

numeric_cols = ['Qty to receive']
df_month_level_receive[numeric_cols] = df_month_level_receive[numeric_cols].round(2).fillna(0).astype(int)
#doubt
#alll columns in numpy
item_codes = df_month_level_receive['Item Code'].to_numpy()
locations = df_month_level_receive['Location'].to_numpy()
delivery_dates = df_month_level_receive['Delivery date'].to_numpy()
delivery_qtys = df_month_level_receive['Qty to receive'].to_numpy()
expiry_dates = df_month_level_receive['Expiry Date'].to_numpy()

# Initialize nested array to hold grouped data
dict_month_level_delivery = {}

# Iterate through unique combinations and group corresponding rows
for i, (item_code, location) in enumerate(df_unique_groups_expiry.itertuples(index=False)):
    mask = (item_codes == item_code) & (locations == location)
    group_data = np.column_stack((
        delivery_dates[mask],
        expiry_dates[mask],
        delivery_qtys[mask]
    ))
    dict_month_level_delivery[(item_code, location)] = group_data

In [1052]:
df_month_level_receive = df_month_level_receive.rename(columns={'Delivery date': 'Month'})

## 2.5 Merge

In [1053]:
df_forecast_output

,Month,Item Code,Location,Best Fit Forecast
0,2024-05,19076008,Aramax Bangalore,0.0
1,2024-05,19076008,Bhiwandi,0.0
2,2024-05,2.01E+11,Aramax Bangalore,0.5
3,2024-05,2.01E+11,Bhiwandi,0.5
4,2024-05,2.01E+12,Bhiwandi,0.57
...,...,...,...,...
4099,2025-10,93030,Bhiwandi,4.48
4100,2025-10,93030,Myntra SJIT - Bangalore,0.43
4101,2025-10,93030,Myntra SJIT - Mumbai,0.0
4102,2025-10,PCSLBG11023,Aramax Bangalore,0.0


In [1054]:
df_exp_qty_with_date.head(25)

,Item Code,Location,Month,Expiry Quantity
0,19076008,Aramax Bangalore,2024-08,0.0
1,19076008,Bhiwandi,2024-08,0.0
2,2.01E+11,Aramax Bangalore,2024-08,0.0
3,2.01E+11,Bhiwandi,2024-08,0.0
4,2.01E+12,Bhiwandi,2024-08,0.0
5,20101900,Aramax Bangalore,2024-08,0.0
6,20101900,Bhiwandi,2024-08,0.0
7,20161907,Aramax Bangalore,2024-08,0.0
8,20161907,Bhiwandi,2024-08,0.0
9,21150,Amazon FBA,2024-07,44.0


In [1055]:
df_final = pd.merge(df_forecast_output, df_exp_qty_with_date, on=['Item Code', 'Location', 'Month'], how='left')
df_final['Expiry Quantity'] = df_final['Expiry Quantity'].fillna(0)

In [1056]:
df_final = pd.merge(df_final, df_item_info, on=['Item Code', 'Location'], how='left')

In [1057]:
df_final['Month'] = pd.to_datetime(df_final['Month'], errors='coerce')
df_intransit['Month'] = pd.to_datetime(df_intransit['Month'], errors='coerce')

df_final = pd.merge(df_final, df_intransit, on=['Item Code', 'Location', 'Month'], how='left')
df_final['Qty to receive'] = df_final['Qty to receive'].fillna(0)

In [1058]:
df_final

,Month,Item Code,Location,Best Fit Forecast,Expiry Quantity,Lead time,Safety Stock Cover,Units per pallet,"Shelf life, days",DC Name,Qty to receive
0,2024-05-01,19076008,Aramax Bangalore,0.0,0.0,1,2,20,120.0,NaN,0.0
1,2024-05-01,19076008,Bhiwandi,0.0,0.0,2,4,20,120.0,NaN,0.0
2,2024-05-01,2.01E+11,Aramax Bangalore,0.5,0.0,1,2,20,120.0,NaN,0.0
3,2024-05-01,2.01E+11,Aramax Bangalore,0.5,0.0,1,2,20,120.0,NaN,0.0
4,2024-05-01,2.01E+11,Bhiwandi,0.5,0.0,2,4,20,120.0,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...
4171,2025-10-01,93030,Bhiwandi,4.48,0.0,2,4,30,120.0,NaN,0.0
4172,2025-10-01,93030,Myntra SJIT - Bangalore,0.43,0.0,1,2,20,120.0,Aramax Bangalore,0.0
4173,2025-10-01,93030,Myntra SJIT - Mumbai,0.0,0.0,1,2,20,120.0,Aramax Bangalore,0.0
4174,2025-10-01,PCSLBG11023,Aramax Bangalore,0.0,0.0,1,2,20,120.0,NaN,0.0


In [1059]:
df_final_dc_not_nan = df_final[df_final['DC Name'].notna()]

df_final_dc_nan = df_final[df_final['DC Name'].isna()]

In [1060]:
def get_final_format(df_final):
    df_item_groups = df_final[['Item Code','Location']]
    df_item_groups = df_item_groups.drop_duplicates()

#time format for numpy
#df_forecast_output['Month'] = df_forecast_output['Month'].dt.strftime('%Y-%m')

    numeric_cols = ['Best Fit Forecast', 'Safety Stock Cover', 'Expiry Quantity', 'Units per pallet', 'Shelf life, days', 'Lead time', 'Qty to receive']
    df_final[numeric_cols] = df_final[numeric_cols].round(2).fillna(0).astype(int)

#alll columns in numpy
    item_codes = df_final['Item Code'].to_numpy()
    locations = df_final['Location'].to_numpy()
    months = df_final['Month'].to_numpy()
    best_fit_forecasts = df_final['Best Fit Forecast'].to_numpy()
    cover_months = df_final['Safety Stock Cover'].to_numpy()
    expired_quantities = df_final['Expiry Quantity'].to_numpy()
    pallet_units = df_final['Units per pallet'].to_numpy()
    shelf_lifes = df_final['Shelf life, days'].to_numpy()
    lead_times = df_final['Lead time'].to_numpy()
    receivables = df_final['Qty to receive'].to_numpy()
# Initialize nested array to hold grouped data
    inventory_np = np.empty((len(df_item_groups),), dtype=object)
#inventory_np = np.array([], dtype=object)

# Iterate through unique combinations and group corresponding rows
    for i, (item_code, location) in enumerate(df_item_groups.itertuples(index=False)):
        mask = (item_codes == item_code) & (locations == location)
        group_data = np.column_stack((
            item_codes[mask],
            locations[mask],
            months[mask],
            best_fit_forecasts[mask],
            cover_months[mask],
            expired_quantities[mask],
            pallet_units[mask],
            shelf_lifes[mask],
            lead_times[mask],
            receivables[mask]
        ))
        inventory_np[i] = np.array(group_data)
    return inventory_np

In [1061]:
def calculate_cover(inventory_np):
    for idx, inv in enumerate(inventory_np):
        if inv is None:
            continue
        cover_months = inv[0,4]
        if cover_months == 0:
            continue
        forecast_values = inv[:,3]
        cumulative_sum_b = np.cumsum(forecast_values)
    
     #   print('cover_months', cover_months)
     #   print('forecast_values', forecast_values)
    
    # Create an array to store the sums for each row in table A
        sums = np.zeros_like(forecast_values)
    
        sums[0] = cumulative_sum_b[cover_months-1]
    
        sums[1:len(forecast_values)-cover_months+1] = cumulative_sum_b[cover_months:] - cumulative_sum_b[:len(forecast_values)-cover_months]
    # Calculate sum for the last cover elements in sums
        sums[len(forecast_values)-cover_months+1:] = cumulative_sum_b[-1] - cumulative_sum_b[len(forecast_values)-cover_months:-1]
        inv[:,4] = sums
    return inventory_np

In [1062]:
def getNextExpiry(group, current_forecast, current_month):
    expiry_np = dict_month_level_expiry.get(group, None)
    expiry_qty = 0
    write_off_qty = 0
    expiry_date = None
    if(expiry_np is None):
        return expiry_date, expiry_qty, write_off_qty
    while(len(expiry_np) > 0 and expiry_qty < current_forecast):
        if int((np.datetime64(expiry_np[0][0]) - np.datetime64(current_month)).astype('timedelta64[D]') / np.timedelta64(1, 'D')) == 0:
            write_off_qty = expiry_np[0][1]
        else:
            expiry_qty = expiry_qty + expiry_np[0][1]
            expiry_date = expiry_np[0][0]
        expiry_np = expiry_np[1:]
        dict_month_level_expiry[group] = dict_month_level_expiry[group][1:]
    return expiry_date, expiry_qty, write_off_qty

def getNextShipment(group, current_forecast, current_month):
    shipment_np = dict_month_level_delivery.get(group, None)
    expiry_date = None
    qty = 0
    write_off_qty = 0
    if(shipment_np is not None and len(shipment_np) > 0):
        while(len(shipment_np) > 0 and qty < current_forecast and  int((np.datetime64(shipment_np[0][0]) - np.datetime64(current_month)).astype('timedelta64[D]') / np.timedelta64(1, 'D')) <= 0):
            if int((np.datetime64(shipment_np[0][1]) - np.datetime64(current_month)).astype('timedelta64[D]') / np.timedelta64(1, 'D')) == 0:
                print('shipment_np: ', shipment_np)
                print('expiry date: ', np.datetime64(shipment_np[0][1]))
                print('current_month: ', np.datetime64(current_month))
                print('date diff: ', int((np.datetime64(shipment_np[0][1]) - np.datetime64(current_month)).astype('timedelta64[D]') / np.timedelta64(1, 'D')))
                write_off_qty = shipment_np[0][2]
            else:
                qty = qty + shipment_np[0][2]
                expiry_date = shipment_np[0][1]
            shipment_np = shipment_np[1:]
            dict_month_level_delivery[group] = dict_month_level_delivery[group][1:]
    return expiry_date, qty, write_off_qty
            
def getNextBatch(group, current_forecast, current_month):
    expiry_date = None
    inv_expiry_qty = 0
    inv_write_off_qty = 0
    del_expiry_qty = 0
    del_write_off = -1 #-1 acts as flag that this is not shipment qty- so put write-offs in inv-write-offs
    
    expiry_date, inv_expiry_qty, inv_write_off_qty = getNextExpiry(group, current_forecast, current_month)
    
    if(inv_expiry_qty < current_forecast):
        expiry_date, del_expiry_qty, del_write_off = getNextShipment(group, current_forecast-inv_expiry_qty, current_month)
        # del_write_off becomes 0 or more here- so put write-offs in shipment-write-offs

    return expiry_date, inv_expiry_qty+del_expiry_qty, inv_write_off_qty, del_write_off

def getExpiryOpenings(inventory_np):
    
    for idx, inv in enumerate(inventory_np):
        if inv is None:
            continue
        group = (inv[0][0], inv[0][1])

        curr_idx = 0
        current_forecast = inv[curr_idx][3]
        current_month = inv[curr_idx][2]

        expiry_date, expiry_qty, inv_write_off_qty, del_write_off_qty = getNextBatch(group, current_forecast, current_month)

        expiry_opening = np.zeros(inv.shape[0], dtype='int64')
        inv_writeoff = np.zeros(inv.shape[0], dtype='int64')
        shipment_inv_writeoff = np.zeros(inv.shape[0], dtype='int64')

        if inv_write_off_qty>0:
            inv_writeoff[curr_idx] = inv_write_off_qty
        if del_write_off_qty>0:
            shipment_inv_writeoff[curr_idx] = del_write_off_qty

        while expiry_date is not None:
            ex_month = np.datetime64(expiry_date)  #expiry date
            current_month = np.datetime64(inv[curr_idx][2]) #current month
            months_before_expiry = min(int((ex_month - current_month).astype('timedelta64[M]').astype(int)), len(expiry_opening)-curr_idx) #how many rows to iterate
            expiry_date = None        
            forecast_np = inv[curr_idx:curr_idx+months_before_expiry,3].astype(int)
            expiry_opening[curr_idx] = expiry_qty
            diff = np.cumsum(-forecast_np) + expiry_qty
            expiry_opening[curr_idx+1:curr_idx+months_before_expiry] = diff[:-1]
            prev_idx = curr_idx
            curr_idx = np.argmax(expiry_opening <= 0)

            if(curr_idx-prev_idx < months_before_expiry):

                curr_idx = curr_idx-1 if expiry_opening[curr_idx] < 0 else curr_idx #we need more qty at curr-1
                print('1- get more qty at idx ',curr_idx)
                expiry_date, expiry_qty, inv_write_off_qty, del_write_off_qty = getNextBatch(group, inv[curr_idx][3] - expiry_opening[curr_idx], inv[curr_idx][2])

                if(inv_write_off_qty>0):
                    print('1- updating inv_write_off_qty', inv_write_off_qty)
                    inv_writeoff[curr_idx] = inv_write_off_qty
                if(del_write_off_qty>0):
                    print('1- updating del_write_off_qty', del_write_off_qty)
                    shipment_inv_writeoff[curr_idx] = del_write_off_qty

                if(expiry_qty>0):
                    expiry_qty = expiry_qty + expiry_opening[curr_idx]
                    continue
                else:
                    print('1- no more qty')
                    curr_idx = curr_idx + 1
            if(curr_idx-prev_idx == months_before_expiry): #completed full month iteration
                if(expiry_opening[curr_idx-1] < inv[curr_idx-1][3]): #opening expiry less than forecast- get more qty
                    print('2- get more qty at idx ',curr_idx-1)
                    curr_idx = curr_idx-1 #we need more qty at curr-1

                    expiry_date, expiry_qty, inv_write_off_qty, del_write_off_qty = getNextBatch(group, inv[curr_idx][3] - expiry_opening[curr_idx], inv[curr_idx][2])

                    if(inv_write_off_qty>0):
                        print('2- updating inv_write_off_qty', inv_write_off_qty)
                        inv_writeoff[curr_idx] = inv_write_off_qty
                    if(del_write_off_qty>0):
                        print('2- updating del_write_off_qty', del_write_off_qty)
                        shipment_inv_writeoff[curr_idx] = del_write_off_qty

                    if(expiry_qty>0):
                        expiry_qty = expiry_qty + expiry_opening[curr_idx]
                        continue
                    else:
                        print('2- no more qty')
                        curr_idx = curr_idx + 1
                else:

                    if(del_write_off_qty < 0):
                        print('3- updating inv_writeoff: ', expiry_opening[curr_idx-1] - inv[curr_idx-1][3])
                        inv_writeoff[curr_idx] = expiry_opening[curr_idx-1] - inv[curr_idx-1][3]
                    else:
                        print('3- del_write_off_qty', expiry_opening[curr_idx-1] - inv[curr_idx-1][3])
                        shipment_inv_writeoff[curr_idx] = expiry_opening[curr_idx-1] - inv[curr_idx-1][3]

                    print('3- get more qty at idx ',curr_idx)

                    expiry_date, expiry_qty, inv_write_off_qty, del_write_off_qty = getNextBatch(group, inv[curr_idx][3], inv[curr_idx][2])

                    if(inv_write_off_qty>0):
                        inv_writeoff[curr_idx] = inv_write_off_qty
                    if(del_write_off_qty>0):
                        shipment_inv_writeoff[curr_idx] = del_write_off_qty

                    if(expiry_qty>0):
                        continue
                    else:
                        print('3- no more qty')
        expiry_opening = expiry_opening.reshape(inv.shape[0],1)
        expiry_opening[expiry_opening<0] = 0
        inv_writeoff = inv_writeoff.reshape(inv.shape[0],1)
        shipment_inv_writeoff = shipment_inv_writeoff.reshape(inv.shape[0],1)
        inv_writeoff[inv_writeoff<0] = 0
        shipment_inv_writeoff[shipment_inv_writeoff<0] = 0
        inv = np.hstack([inv, expiry_opening, inv_writeoff, shipment_inv_writeoff])
        inventory_np[idx] = inv
        # Define column names for the DataFrame
        #column_names = ['Item Code', 'Location', 'Month', 'Forecast', 'Cover', 'Expired Qty', 'Units per pallet', 'Shelf life, days', 'Lead time', 'Receivables', 'Expiry Opening', 'Write-offs', 'Shipment write-offs']

        # Create DataFrame
        #df_show = pd.DataFrame(inventory_np[idx], columns=column_names)



        #print(df_show.to_string(index=False))
        #print('***************')
    return inventory_np

In [1063]:
def getWriteOffs(inventory_np):
    inventory_np = np.array(inventory_np.tolist())
    #print(inventory_np.shape)

    #Expiry Opening-9 || Write-offs-10 || Shipment write-offs-11
    opening_inventory = np.zeros((inventory_np.shape[0], inventory_np[0].shape[0], 1), dtype='int64') #13
    projected_arrival = np.zeros((inventory_np.shape[0], inventory_np[0].shape[0], 1), dtype='int64') #14
    arrival_write_off = np.zeros((inventory_np.shape[0], inventory_np[0].shape[0], 1), dtype='int64') #15
    order_wo_pallet = np.zeros((inventory_np.shape[0], inventory_np[0].shape[0], 1), dtype='int64') #16
    projected_order = np.zeros((inventory_np.shape[0], inventory_np[0].shape[0], 1), dtype='int64') #17


    inventory_np = np.concatenate((inventory_np, opening_inventory, projected_arrival, arrival_write_off, order_wo_pallet, projected_order), axis=-1)

    #print(inventory_np.shape)

    # Modify the 12th column by adding the respective 1st column to it
    codes = inventory_np[:, 0, 0].astype(str)
    locations = inventory_np[:, 0, 1].astype(str)
    #print('codes', codes)
    #print('locations', locations)
    # Loop over each element in the first dimension
    for i in range(inventory_np.shape[0]):
        key_loh = (codes[i], locations[i])
        # Perform the calculation and assignment
     #   print(key_loh, dict_loh.get(key_loh, 0))
        inventory_np[i, 0, 13] = np.maximum(0, (
            dict_loh.get(key_loh, 0)
            + inventory_np[i, 0, 9]
            - inventory_np[i, 0, 11] #writeoffs
            - inventory_np[i, 0, 12] #shipment writeoff
        ))

    # Initialize the array with a placeholder for empty dates
    empty_placeholder = "0000-00"
    expiry_tracker_date = np.full((inventory_np.shape[0],1), empty_placeholder, dtype=object)
    expiry_tracker_qty = np.zeros((inventory_np.shape[0],1), dtype='int64')

    #opening_inventory: 13, projected_arrival: 14, arrival_write_off: 15, order_wo_pallet: 16, projected_order: 17
    for i in range(1, inventory_np.shape[1]):
        inventory_np[:,i,13] = np.maximum(0,(inventory_np[:,i-1,13] #previous
                               + inventory_np[:,i,9]
                               - inventory_np[:,i,11] #write-off
                               - inventory_np[:,i,12] #transit-writeoff
                               - inventory_np[:,i-1,3])) #forecast

        #check for arrival writeoffs
        expired_idx = inventory_np[:, i, 2] == expiry_tracker_date[:,0]
        #print('expired_idx', expired_idx)
        inventory_np[expired_idx, i, 15] = np.maximum(0, inventory_np[expired_idx, i-1, 13]-inventory_np[expired_idx, i-1, 3])

        #include arrival writeoff in opening inventory
        inventory_np[expired_idx, i, 13] -= inventory_np[expired_idx, i, 15]

        # get lead time
        lead_time = inventory_np[:, i, 8]
        lead_time_check = (i - lead_time).astype(int) >= 0
        cover_check = inventory_np[:, i, 13] < inventory_np[:, i, 4]
        # Check if inventory below cover
        below_threshold = cover_check & lead_time_check #cover

        #place order T-lead-time for indexes below cover
        indices_to_place_order = (i - lead_time[below_threshold]).astype(int)

        #order without pallet at index i - lead time
        inventory_np[below_threshold, indices_to_place_order, 16] = inventory_np[below_threshold, i, 4] - inventory_np[below_threshold, i, 13]

        #get qty to order based on pallet units
        qty_to_order = np.ceil(inventory_np[below_threshold, indices_to_place_order, 16]/inventory_np[below_threshold, indices_to_place_order, 6])*inventory_np[below_threshold, indices_to_place_order, 6]

        #exact order place at index i-lead time
        inventory_np[below_threshold, indices_to_place_order, 17] = qty_to_order

        #orders arrive at index i
        inventory_np[below_threshold, i, 14] = qty_to_order

        inventory_np[below_threshold, i, 13] += qty_to_order 

        #store expiry dates and qty when orders arrive
        expiry_tracker_qty[below_threshold, 0] = qty_to_order

        order_arrival_dates = inventory_np[below_threshold, i, 2].astype('datetime64[M]')
        shelf_life = (inventory_np[below_threshold, i, 7]//30).astype('timedelta64[M]')
        expiry_dates = order_arrival_dates + shelf_life
        expiry_dates_str = expiry_dates.astype('datetime64[M]').astype(str)
        #print('current month ', inventory_np[:, i, 2])
        #print('expiry_dates_str ', expiry_dates_str)
        if expiry_dates_str.size > 0:
            expiry_tracker_date[below_threshold, 0] = expiry_dates_str
            #print('expiry_tracker_date ',expiry_tracker_date)

    column_names = (['Item_Code', 'Location', 'Month', 
                         'Forecast', 'Cover', 'Expired_Qty', 
                         'Units_per_pallet', 'Shelf_life', 
                         'Lead_time', 'Receivables', 'Expiry_Opening', 'Write-offs', 
                         'Shipment_write-offs','opening_inventory', 
                         'projected_arrival', 'arrival_write_off', 
                         'order_wo_pallet', 'projected_order'])

    #for idx, inv in enumerate(inventory_np):
    # Define column names for the DataFrame


            # Create DataFrame
     #   df_show = pd.DataFrame(inventory_np[idx], columns=column_names)



    #    print(df_show.to_string(index=False))
       # print('***************')
    return inventory_np

In [1064]:
inventory_np_with_dc = get_final_format(df_final_dc_not_nan)
inventory_np_no_dc = get_final_format(df_final_dc_nan)

C:\Users\Anamind\AppData\Local\Temp\ipykernel_21924\680522246.py:9: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_final[numeric_cols] = df_final[numeric_cols].round(2).fillna(0).astype(int)
C:\Users\Anamind\AppData\Local\Temp\ipykernel_21924\680522246.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final[numeric_cols] = df_final[numeric_cols].round(2).fillna(0).astype(int)
C:\Users\Anamind\AppData\Local\Temp\ipykernel_21924\680522246.py:9: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is depre

In [1065]:
inventory_np_with_dc

array([array([['21150', 'Amazon FBA', 1714521600000000000, 0, 4, 0, 30, 120, 1,
               0],
              ['21150', 'Amazon FBA', 1717200000000000000, 0, 4, 0, 30, 120, 1,
               0],
              ['21150', 'Amazon FBA', 1719792000000000000, 0, 4, 44, 30, 120, 1,
               0],
              ['21150', 'Amazon FBA', 1722470400000000000, 1, 4, 0, 30, 120, 1,
               0],
              ['21150', 'Amazon FBA', 1725148800000000000, 1, 4, 0, 30, 120, 1,
               0],
              ['21150', 'Amazon FBA', 1727740800000000000, 1, 4, 0, 30, 120, 1,
               0],
              ['21150', 'Amazon FBA', 1730419200000000000, 1, 4, 0, 30, 120, 1,
               0],
              ['21150', 'Amazon FBA', 1733011200000000000, 1, 4, 0, 30, 120, 1,
               0],
              ['21150', 'Amazon FBA', 1735689600000000000, 1, 4, 0, 30, 120, 1,
               0],
              ['21150', 'Amazon FBA', 1738368000000000000, 1, 4, 0, 30, 120, 1,
               0],
         

In [1066]:
df_final_dc_nan

,Month,Item Code,Location,Best Fit Forecast,Expiry Quantity,Lead time,Safety Stock Cover,Units per pallet,"Shelf life, days",DC Name,Qty to receive
0,2024-05-01,19076008,Aramax Bangalore,0,0,1,2,20,120,NaN,0
1,2024-05-01,19076008,Bhiwandi,0,0,2,4,20,120,NaN,0
2,2024-05-01,2.01E+11,Aramax Bangalore,0,0,1,2,20,120,NaN,0
3,2024-05-01,2.01E+11,Aramax Bangalore,0,0,1,2,20,120,NaN,0
4,2024-05-01,2.01E+11,Bhiwandi,0,0,2,4,20,120,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...
4168,2025-10-01,79807750,Bhiwandi,19,0,2,4,20,120,NaN,0
4170,2025-10-01,93030,Aramax Bangalore,2,0,1,2,30,120,NaN,0
4171,2025-10-01,93030,Bhiwandi,4,0,2,4,30,120,NaN,0
4174,2025-10-01,PCSLBG11023,Aramax Bangalore,0,0,1,2,20,120,NaN,0


In [1067]:
df_final_dc_not_nan

,Month,Item Code,Location,Best Fit Forecast,Expiry Quantity,Lead time,Safety Stock Cover,Units per pallet,"Shelf life, days",DC Name,Qty to receive
11,2024-05-01,21150,Amazon FBA,0,0,1,4,30,120,Bhiwandi,0
14,2024-05-01,21150,Myntra SJIT - Bangalore,2,0,1,4,20,120,Aramax Bangalore,0
15,2024-05-01,21150,Myntra SJIT - Mumbai,0,0,1,4,20,120,Aramax Bangalore,0
16,2024-05-01,21351,Amazon FBA,5,0,1,4,30,120,Bhiwandi,0
19,2024-05-01,21351,Myntra SJIT - Bangalore,4,0,1,4,20,120,Aramax Bangalore,0
...,...,...,...,...,...,...,...,...,...,...,...
4154,2025-10-01,29560,Myntra SJIT - Bangalore,1,0,1,2,20,120,Aramax Bangalore,0
4155,2025-10-01,29560,Myntra SJIT - Mumbai,1,0,1,2,20,120,Aramax Bangalore,0
4169,2025-10-01,93030,Amazon FBA,1,0,1,2,30,120,Bhiwandi,0
4172,2025-10-01,93030,Myntra SJIT - Bangalore,0,0,1,2,20,120,Aramax Bangalore,0


In [1068]:
inventory_np_no_dc

array([array([['19076008', 'Aramax Bangalore', 1714521600000000000, 0, 2, 0, 20,
               120, 1, 0],
              ['19076008', 'Aramax Bangalore', 1717200000000000000, 0, 2, 0, 20,
               120, 1, 0],
              ['19076008', 'Aramax Bangalore', 1719792000000000000, 0, 2, 0, 20,
               120, 1, 0],
              ['19076008', 'Aramax Bangalore', 1722470400000000000, 0, 2, 0, 20,
               120, 1, 0],
              ['19076008', 'Aramax Bangalore', 1725148800000000000, 0, 2, 0, 20,
               120, 1, 0],
              ['19076008', 'Aramax Bangalore', 1727740800000000000, 0, 2, 0, 20,
               120, 1, 0],
              ['19076008', 'Aramax Bangalore', 1730419200000000000, 0, 2, 0, 20,
               120, 1, 0],
              ['19076008', 'Aramax Bangalore', 1733011200000000000, 0, 2, 0, 20,
               120, 1, 0],
              ['19076008', 'Aramax Bangalore', 1735689600000000000, 0, 2, 0, 20,
               120, 1, 0],
              ['19076008', '

In [1069]:
df_final_dc_not_nan

,Month,Item Code,Location,Best Fit Forecast,Expiry Quantity,Lead time,Safety Stock Cover,Units per pallet,"Shelf life, days",DC Name,Qty to receive
11,2024-05-01,21150,Amazon FBA,0,0,1,4,30,120,Bhiwandi,0
14,2024-05-01,21150,Myntra SJIT - Bangalore,2,0,1,4,20,120,Aramax Bangalore,0
15,2024-05-01,21150,Myntra SJIT - Mumbai,0,0,1,4,20,120,Aramax Bangalore,0
16,2024-05-01,21351,Amazon FBA,5,0,1,4,30,120,Bhiwandi,0
19,2024-05-01,21351,Myntra SJIT - Bangalore,4,0,1,4,20,120,Aramax Bangalore,0
...,...,...,...,...,...,...,...,...,...,...,...
4154,2025-10-01,29560,Myntra SJIT - Bangalore,1,0,1,2,20,120,Aramax Bangalore,0
4155,2025-10-01,29560,Myntra SJIT - Mumbai,1,0,1,2,20,120,Aramax Bangalore,0
4169,2025-10-01,93030,Amazon FBA,1,0,1,2,30,120,Bhiwandi,0
4172,2025-10-01,93030,Myntra SJIT - Bangalore,0,0,1,2,20,120,Aramax Bangalore,0


In [1070]:
inventory_np_with_dc = calculate_cover(inventory_np_with_dc)

In [1071]:
inventory_np_with_dc = getExpiryOpenings(inventory_np_with_dc)

In [1072]:
inventory_np_with_dc = getWriteOffs(inventory_np_with_dc)
inventory_np_with_dc

array([[['21150', 'Amazon FBA', 1714521600000000000, ..., 0, 0, 0],
        ['21150', 'Amazon FBA', 1717200000000000000, ..., 0, 0, 0],
        ['21150', 'Amazon FBA', 1719792000000000000, ..., 0, 0, 0],
        ...,
        ['21150', 'Amazon FBA', 1754006400000000000, ..., 0, 0, 0],
        ['21150', 'Amazon FBA', 1756684800000000000, ..., 0, 0, 0],
        ['21150', 'Amazon FBA', 1759276800000000000, ..., 0, 0, 0]],

       [['21150', 'Myntra SJIT - Bangalore', 1714521600000000000, ...,
         0, 12, 20],
        ['21150', 'Myntra SJIT - Bangalore', 1717200000000000000, ...,
         0, 0, 0],
        ['21150', 'Myntra SJIT - Bangalore', 1719792000000000000, ...,
         0, 0, 0],
        ...,
        ['21150', 'Myntra SJIT - Bangalore', 1754006400000000000, ...,
         0, 0, 0],
        ['21150', 'Myntra SJIT - Bangalore', 1756684800000000000, ...,
         0, 0, 0],
        ['21150', 'Myntra SJIT - Bangalore', 1759276800000000000, ...,
         0, 0, 0]],

       [['21150', 'M

In [1073]:
inventory_np_with_dc

array([[['21150', 'Amazon FBA', 1714521600000000000, ..., 0, 0, 0],
        ['21150', 'Amazon FBA', 1717200000000000000, ..., 0, 0, 0],
        ['21150', 'Amazon FBA', 1719792000000000000, ..., 0, 0, 0],
        ...,
        ['21150', 'Amazon FBA', 1754006400000000000, ..., 0, 0, 0],
        ['21150', 'Amazon FBA', 1756684800000000000, ..., 0, 0, 0],
        ['21150', 'Amazon FBA', 1759276800000000000, ..., 0, 0, 0]],

       [['21150', 'Myntra SJIT - Bangalore', 1714521600000000000, ...,
         0, 12, 20],
        ['21150', 'Myntra SJIT - Bangalore', 1717200000000000000, ...,
         0, 0, 0],
        ['21150', 'Myntra SJIT - Bangalore', 1719792000000000000, ...,
         0, 0, 0],
        ...,
        ['21150', 'Myntra SJIT - Bangalore', 1754006400000000000, ...,
         0, 0, 0],
        ['21150', 'Myntra SJIT - Bangalore', 1756684800000000000, ...,
         0, 0, 0],
        ['21150', 'Myntra SJIT - Bangalore', 1759276800000000000, ...,
         0, 0, 0]],

       [['21150', 'M

In [1074]:
reshaped_data = inventory_np_with_dc.reshape(-1, inventory_np_with_dc.shape[2])


In [1075]:
column_names = (['Item_Code', 'Location', 'Month', 
                         'Forecast', 'Cover', 'Expired_Qty', 
                         'Units_per_pallet', 'Shelf_life', 
                         'Lead_time', 'Receivables', 'Expiry_Opening', 'Write-offs', 
                         'Shipment_write-offs','opening_inventory', 
                         'projected_arrival', 'arrival_write_off', 
                         'order_wo_pallet', 'projected_order'])


In [1076]:
column_names

['Item_Code',
 'Location',
 'Month',
 'Forecast',
 'Cover',
 'Expired_Qty',
 'Units_per_pallet',
 'Shelf_life',
 'Lead_time',
 'Receivables',
 'Expiry_Opening',
 'Write-offs',
 'Shipment_write-offs',
 'opening_inventory',
 'projected_arrival',
 'arrival_write_off',
 'order_wo_pallet',
 'projected_order']

In [1077]:
# Create a DataFrame
merged_data = pd.DataFrame(reshaped_data, columns=column_names)

In [1078]:
merged_data

,Item_Code,Location,Month,Forecast,Cover,Expired_Qty,Units_per_pallet,Shelf_life,Lead_time,Receivables,Expiry_Opening,Write-offs,Shipment_write-offs,opening_inventory,projected_arrival,arrival_write_off,order_wo_pallet,projected_order
0,21150,Amazon FBA,1714521600000000000,0,1,0,30,120,1,0,0,0,0,10,0,0,0,0
1,21150,Amazon FBA,1717200000000000000,0,2,0,30,120,1,0,0,0,0,10,0,0,0,0
2,21150,Amazon FBA,1719792000000000000,0,3,44,30,120,1,0,0,0,0,10,0,0,0,0
3,21150,Amazon FBA,1722470400000000000,1,4,0,30,120,1,0,0,0,0,10,0,0,0,0
4,21150,Amazon FBA,1725148800000000000,1,4,0,30,120,1,0,0,0,0,9,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2191,93030,Myntra SJIT - Mumbai,1748736000000000000,0,0,0,20,120,1,0,0,0,0,0,0,0,0,0
2192,93030,Myntra SJIT - Mumbai,1751328000000000000,0,0,0,20,120,1,0,0,0,0,0,0,0,0,0
2193,93030,Myntra SJIT - Mumbai,1754006400000000000,0,0,0,20,120,1,0,0,0,0,0,0,0,0,0
2194,93030,Myntra SJIT - Mumbai,1756684800000000000,0,0,0,20,120,1,0,0,0,0,0,0,0,0,0


In [1079]:
print("Shape of reshaped_data:", reshaped_data.shape)
print("Number of column names:", len(column_names))

Shape of reshaped_data: (2196, 18)
Number of column names: 18


In [1080]:
inventory_np_with_dc

array([[['21150', 'Amazon FBA', 1714521600000000000, ..., 0, 0, 0],
        ['21150', 'Amazon FBA', 1717200000000000000, ..., 0, 0, 0],
        ['21150', 'Amazon FBA', 1719792000000000000, ..., 0, 0, 0],
        ...,
        ['21150', 'Amazon FBA', 1754006400000000000, ..., 0, 0, 0],
        ['21150', 'Amazon FBA', 1756684800000000000, ..., 0, 0, 0],
        ['21150', 'Amazon FBA', 1759276800000000000, ..., 0, 0, 0]],

       [['21150', 'Myntra SJIT - Bangalore', 1714521600000000000, ...,
         0, 12, 20],
        ['21150', 'Myntra SJIT - Bangalore', 1717200000000000000, ...,
         0, 0, 0],
        ['21150', 'Myntra SJIT - Bangalore', 1719792000000000000, ...,
         0, 0, 0],
        ...,
        ['21150', 'Myntra SJIT - Bangalore', 1754006400000000000, ...,
         0, 0, 0],
        ['21150', 'Myntra SJIT - Bangalore', 1756684800000000000, ...,
         0, 0, 0],
        ['21150', 'Myntra SJIT - Bangalore', 1759276800000000000, ...,
         0, 0, 0]],

       [['21150', 'M

In [1081]:
inventory_np_with_dc

array([[['21150', 'Amazon FBA', 1714521600000000000, ..., 0, 0, 0],
        ['21150', 'Amazon FBA', 1717200000000000000, ..., 0, 0, 0],
        ['21150', 'Amazon FBA', 1719792000000000000, ..., 0, 0, 0],
        ...,
        ['21150', 'Amazon FBA', 1754006400000000000, ..., 0, 0, 0],
        ['21150', 'Amazon FBA', 1756684800000000000, ..., 0, 0, 0],
        ['21150', 'Amazon FBA', 1759276800000000000, ..., 0, 0, 0]],

       [['21150', 'Myntra SJIT - Bangalore', 1714521600000000000, ...,
         0, 12, 20],
        ['21150', 'Myntra SJIT - Bangalore', 1717200000000000000, ...,
         0, 0, 0],
        ['21150', 'Myntra SJIT - Bangalore', 1719792000000000000, ...,
         0, 0, 0],
        ...,
        ['21150', 'Myntra SJIT - Bangalore', 1754006400000000000, ...,
         0, 0, 0],
        ['21150', 'Myntra SJIT - Bangalore', 1756684800000000000, ...,
         0, 0, 0],
        ['21150', 'Myntra SJIT - Bangalore', 1759276800000000000, ...,
         0, 0, 0]],

       [['21150', 'M

In [1082]:
merged_data['Month'] = pd.to_datetime(merged_data['Month'], unit='ns').dt.strftime('%Y-%m-%d')

In [1083]:
# Convert 'Month' column to datetime in both DataFrames
df_final_dc_nan['Month'] = pd.to_datetime(df_final_dc_nan['Month'], errors='coerce')
df_final_dc_not_nan['Month'] = pd.to_datetime(df_final_dc_not_nan['Month'], errors='coerce')
merged_data['Month'] = pd.to_datetime(merged_data['Month'], errors='coerce')

# Now perform the merge operation
merged_data = merged_data.merge(
    df_final_dc_not_nan[['Item Code', 'Location', 'Month', 'DC Name']],
    left_on=['Item_Code', 'Location', 'Month'],
    right_on=['Item Code', 'Location', 'Month'],
    how='left'
)

# Drop the 'Item Code' column from df_final_dc_not_nan that was added during the merge
merged_data.drop(columns=['Item Code'], inplace=True)


C:\Users\Anamind\AppData\Local\Temp\ipykernel_21924\3326990050.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final_dc_nan['Month'] = pd.to_datetime(df_final_dc_nan['Month'], errors='coerce')
C:\Users\Anamind\AppData\Local\Temp\ipykernel_21924\3326990050.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final_dc_not_nan['Month'] = pd.to_datetime(df_final_dc_not_nan['Month'], errors='coerce')


In [1084]:
#df_final_dc_nan['Month'] = pd.to_datetime(df_forecast_output2['Month'])
merged_data['Month'] = pd.to_datetime(merged_data['Month'])
merged_df =df_final_dc_nan.merge(merged_data[['Item_Code', 'DC Name', 'Month', 'projected_order']], 
                                      left_on=['Item Code', 'Location', 'Month'], 
                                      right_on=['Item_Code', 'DC Name', 'Month'], 
                                      how='left')

# Add the 'projected_order' column values to the 'Forecast' column
merged_df['Best Fit Forecast'] = merged_df['Best Fit Forecast'] + merged_df['projected_order'].fillna(0)

# Drop the 'projected_order' column as it's no longer needed
merged_df.drop(columns=['projected_order'], inplace=True)



C:\Users\Anamind\AppData\Local\Temp\ipykernel_21924\3264847738.py:9: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  merged_df['Best Fit Forecast'] = merged_df['Best Fit Forecast'] + merged_df['projected_order'].fillna(0)


In [1085]:
df_final_dc_nan

,Month,Item Code,Location,Best Fit Forecast,Expiry Quantity,Lead time,Safety Stock Cover,Units per pallet,"Shelf life, days",DC Name,Qty to receive
0,2024-05-01,19076008,Aramax Bangalore,0,0,1,2,20,120,NaN,0
1,2024-05-01,19076008,Bhiwandi,0,0,2,4,20,120,NaN,0
2,2024-05-01,2.01E+11,Aramax Bangalore,0,0,1,2,20,120,NaN,0
3,2024-05-01,2.01E+11,Aramax Bangalore,0,0,1,2,20,120,NaN,0
4,2024-05-01,2.01E+11,Bhiwandi,0,0,2,4,20,120,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...
4168,2025-10-01,79807750,Bhiwandi,19,0,2,4,20,120,NaN,0
4170,2025-10-01,93030,Aramax Bangalore,2,0,1,2,30,120,NaN,0
4171,2025-10-01,93030,Bhiwandi,4,0,2,4,30,120,NaN,0
4174,2025-10-01,PCSLBG11023,Aramax Bangalore,0,0,1,2,20,120,NaN,0


In [1086]:
inventory_np_no_dc = get_final_format(df_final_dc_nan)

C:\Users\Anamind\AppData\Local\Temp\ipykernel_21924\680522246.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final[numeric_cols] = df_final[numeric_cols].round(2).fillna(0).astype(int)


In [1087]:
inventory_np_no_dc

array([array([['19076008', 'Aramax Bangalore', 1714521600000000000, 0, 2, 0, 20,
               120, 1, 0],
              ['19076008', 'Aramax Bangalore', 1717200000000000000, 0, 2, 0, 20,
               120, 1, 0],
              ['19076008', 'Aramax Bangalore', 1719792000000000000, 0, 2, 0, 20,
               120, 1, 0],
              ['19076008', 'Aramax Bangalore', 1722470400000000000, 0, 2, 0, 20,
               120, 1, 0],
              ['19076008', 'Aramax Bangalore', 1725148800000000000, 0, 2, 0, 20,
               120, 1, 0],
              ['19076008', 'Aramax Bangalore', 1727740800000000000, 0, 2, 0, 20,
               120, 1, 0],
              ['19076008', 'Aramax Bangalore', 1730419200000000000, 0, 2, 0, 20,
               120, 1, 0],
              ['19076008', 'Aramax Bangalore', 1733011200000000000, 0, 2, 0, 20,
               120, 1, 0],
              ['19076008', 'Aramax Bangalore', 1735689600000000000, 0, 2, 0, 20,
               120, 1, 0],
              ['19076008', '

In [1088]:
inventory_np_no_dc = calculate_cover(inventory_np_no_dc)

In [1089]:
inventory_np_no_dc = getExpiryOpenings(inventory_np_no_dc)
inventory_np_no_dc = getWriteOffs(inventory_np_no_dc)

ValueError: Converting an integer to a NumPy datetime requires a specified unit

In [ ]:
columns = (['Item_Code', 'Location', 'Month', 
                     'Forecast', 'Cover', 'Expired_Qty', 
                     'Units_per_pallet', 'Shelf_life', 
                     'Lead_time', 'Receivables', 'Expiry_Opening', 'Write-offs', 
                     'Shipment_write-offs','opening_inventory', 
                     'projected_arrival', 'arrival_write_off', 
                     'order_wo_pallet', 'projected_order'])
